## Describe the environment in the Nim learning model.

The environment of Nim is the board itself, though it should be noted that the board can be a 

tabletop, a book, or anything flat that can adequately support the items used to play Nim.

Ref: Graham, Paul, Lord William, Reinforcement Learning and the Game of Nim, Stockholm Sweden, 2015.
accessed: http://www.diva-portal.org/smash/get/diva2:814832/FULLTEXT01.pdf

## Describe the agent(s) in the Nim learning model (Hint, not just the Q-learner).

1. Random player
Random player is an agent that moves stochastically and does not learn from the environment.


2. Guru player

Guru player is an agent that moves according to a known solution to the Nim game and does not

need to learn.


3. Q-learner

The Q-learner player is an agent that moves according to a learned Q-table generated by the 

interaction of the player with the envionment, i.e. state and action pairs.

## Describe the reward and penalty in the Nim learning model.

The end state for the winning agent to be in is the state where the agent takes the last items of 

the last pile, which I will call the best state. The Q-table is updated with the reward for each 

action that the agent makes for this version of Nim. If the agent arrives in 

the best state, then the Q-table the agent will be rewarded with 100.0, otherwise the reward is

zero, and the Q-table is updated accordingly. I do not see an explicit penalty for not arriving in 

the best state, unless the penalty is getting zero reward, which I would argue is not a penalty, 

since a reward is not deducted.

## How many possible states there could be in the Nim game with a maximum of 10 items per pile and 3 piles total?

According to the game rules, a player can take any number of items from one to ten from one of any 

of the piles at a time. In addition, the player can choose to not take any of the items for one turn

per pile. Translating to reinforcement learning; there are eleven states for each pile, because the 

the size of the Nim state space for each pile is number of items in a given pile (item_num) + the 

one time chance to not take any pieces (1). An equation that can be used to determine the  number of 

states in this cas is num_states_per_pile = item_num + 1. The equation for all of the states is the 

num_states_per_pile to the power of the number of piles (pile_num), which can be written as 

all_possible_states_num = (num_states_per_pile) ^ pile_num = (10 + 1) ^ 3 = 11 ^ 3 = 1331. 

## How many possible unique actions there could be for player 1 in the Nim game with 10 items per pile and 3 piles total?

The player can take any number of items from the pile to include taking no items. Because the player 

can only choose to take 0 items per pile once, it is tempting to add the state of not taking an 

item, but it seems that this line of thinking is false, since the state prior to action is equal to 

the post-action state, therefore, it should not be added as a unique action.

Representing this mathematically, and borrowing the variable developed in problem 4, we arrive at 

num_states_player_1 = num_states_per_pile1 + num_states_per_pile2 + num_state_per_pile3 = 10 + 10 + 10 = 30.

## Find a way to improve the provided Nim game learning model. Do you think onecan beat the Guru player? (Hint: How about penalizing the losses? Hint: It is indeed possible to find a better solution, which improves the way Q-learning updates its Q-table).

# Note: Do not use a Guru player inside the learning module as this would defeat the purpose of reinforcement learning (why?). However, playing with a Guru is permitted.

The reason that putting a guru player inside the learning module is that the guru acts according to the known solution, whereas the Q-Learner does not know the solution, but learns from the environment given a set of rules. 

I hypothesize the following:

1. Training the Q-learner more then the 1000 iterations that it was set to train by default will

increase the Q-learners performance.


__Test 1__: Train the learner with the default 1000 iterations and have Q-learner play against the 

random and guru agents for 1000 games, then repeat with 10,000 training iterations and compare 

results.


2. Penalizing each move that the Q-learner makes will improve the Q-learner's performance. This 

hypothesis might only be true for a range of penalty values. 


__Test 2__: Set a negative penalty value (modify code in nim_qlearn)for each move that the Q-leaner 

makes that lands the agent in a non-winning state. This can be any value that shows a positive 

increase in the performance of the Q-learner.


3. The Q-learner will perform better by using the methods of hypothesis 1 and hypothesis 2 combined. 


__Test 3__: Train the Q-learner play at 10,000 training iterations with a negative penalty for each 

action that the learner takes that lands it in a non-winning state.

In [1]:
'''
Note this code is taken from the module notebook: module09_reinforcement_notebook.html with 
minor modifications.

Initialize the game and define the agents.
'''

import numpy as np
import pandas as pd
from random import randint, choice
import warnings
warnings.filterwarnings('ignore')
# max number of items per pile
ITEMS_MX = 10

# Initialize starting position
def init_game():
    return [randint(1,ITEMS_MX), randint(1,ITEMS_MX), randint(1,ITEMS_MX)]

# Based on X-oring the item counts in piles - mathematical solution
def nim_guru(st):
    xored = st[0] ^ st[1] ^ st[2]
    if xored == 0:
        return nim_random(st)
    #
    for pile in range(3):
        s = st[pile] ^ xored
        if s <= st[pile]:
            return st[pile]-s, pile

# Random Nim player
def nim_random(_st):
    pile = choice([i for i in range(3) if _st[i]>0])  # find the non-empty piles
    return randint(1, _st[pile]), pile  # random move

def nim_qlearner(_st):
    # pick the best rewarding move, equation 1
    a = np.argmax(qtable[_st[0], _st[1], _st[2]])  # exploitation
    # index is based on move, pile
    move, pile = a%ITEMS_MX+1, a//ITEMS_MX
    # check if qtable has generated a random but game illegal move - we have not explored there yet
    if move <= 0 or _st[pile] < move:
        move, pile = nim_random(_st)  # exploration
    return move, pile  # action

Engines = {'Random':nim_random, 'Guru':nim_guru, 'Qlearner':nim_qlearner}

def game(a, b):
    state, side = init_game(), 'A'
    while True:
        engine = Engines[a] if side == 'A' else Engines[b]
        move, pile = engine(state)
        state[pile] -= move
        if state == [0, 0, 0]:  # game ends
            return side  # winning side

        side = 'B' if side == 'A' else 'A'  # switch sides

def play_games(_n, a, b):
    from collections import defaultdict
    wins = defaultdict(int)
    for i in range(_n):
        wins[game(a, b)] += 1
    return wins['A'], wins['B']

In [2]:
'''
Iterate through user-defined number of games and update the Q-learners Q-table

Note this code is taken from the module notebook: module09_reinforcement_notebook.html with 
minor modifications.
'''

qtable, Alpha, Gamma, Reward = None, 0.9, 0.8, 100.0

# learn from _n games, randomly played to explore the possible states
def nim_qlearn(_n, penalty):
    global qtable
    # based on max items per pile
    qtable = np.zeros((ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX*3), dtype=float)
    # play _n games
    for i in range(_n):
        # first state is starting position
        st1 = init_game()
        while True:  # while game not finished
            # make a random move - exploration
            move, pile = nim_random(st1)
            st2 = list(st1)
            # make the move
            st2[pile] -= move  # --> last move I made
            if st2 == [0, 0, 0]:  # game ends
                qtable_update(Reward, st1, move, pile, 0)  # I won
                break  # new game

            qtable_update(penalty, st1, move, pile, np.max(qtable[st2[0], st2[1], st2[2]]))
            st1 = st2

# Equation 3 - update the qtable
def qtable_update(r, _st1, move, pile, q_future_best):
    a = pile*ITEMS_MX+move-1
    qtable[_st1[0], _st1[1], _st1[2], a] = Alpha * (r + Gamma * q_future_best)

## Baseline: The Qlearner learns from the default 1000 iterations of training, and all learners play 1000 games. There are no penalties for losing. 

In [3]:
%%time
##teach the q-learner by playing the game 
learn_num_games = 1000
play_num_games = 1000
explore_penalty = 0
nim_qlearn(learn_num_games, explore_penalty)

##list of different agents
agents = ('Random', 'Guru', 'Qlearner')

##keep track of games
results = {'agent': [], 'score': [], 'turn': [], 'descr': []}

##iterate through agents for the first player to go
print('\nBaseline Game Results')
print('---------------------------------')
for agent1 in agents:
    ##iterate through agents for the second player to go
    for agent2 in agents:
        game_agents = agent1 + '_vs_' + agent2
        a1_score, a2_score = play_games(play_num_games, agent1, agent2)
        
        ##record agent1 performance
        results['agent'].append(agent1)
        results['score'].append(a1_score)
        results['turn'].append('first')
        results['descr'].append('baseline')
        
        ##record agent2 performance
        results['agent'].append(agent2)
        results['score'].append(a2_score)
        results['turn'].append('second')
        results['descr'].append('baseline')
        
        print('\nAgent1: ',agent1, 'Score: ', a1_score)
        print('Agent2: ', agent2, 'score: ', a2_score)
print('\n')


Baseline Game Results
---------------------------------

Agent1:  Random Score:  510
Agent2:  Random score:  490

Agent1:  Random Score:  9
Agent2:  Guru score:  991

Agent1:  Random Score:  316
Agent2:  Qlearner score:  684

Agent1:  Guru Score:  1000
Agent2:  Random score:  0

Agent1:  Guru Score:  935
Agent2:  Guru score:  65

Agent1:  Guru Score:  999
Agent2:  Qlearner score:  1

Agent1:  Qlearner Score:  692
Agent2:  Random score:  308

Agent1:  Qlearner Score:  14
Agent2:  Guru score:  986

Agent1:  Qlearner Score:  552
Agent2:  Qlearner score:  448


CPU times: total: 156 ms
Wall time: 153 ms


## Qlearner train for 10,000 iterations.

In [4]:
%%time
##teach the q-learner by playing the game 
learn_num_games = 10000
play_num_games = 1000
explore_penalty = 0
nim_qlearn(learn_num_games, explore_penalty)

##iterate through agents for the first player to go
print('\nQlearner 10,000 Train Iters Game Results')
print('---------------------------------')
for agent1 in agents:
    ##iterate through agents for the second player to go
    for agent2 in agents:
        a1_score, a2_score = play_games(play_num_games, agent1, agent2)
        
        ##record agent1 performance
        results['agent'].append(agent1)
        results['score'].append(a1_score)
        results['turn'].append('first')
        results['descr'].append('base_increased_train_it')
        
        ##record agent2 performance
        results['agent'].append(agent2)
        results['score'].append(a2_score)
        results['turn'].append('second')
        results['descr'].append('base_increased_train_it')
        
        print('\nAgent1: ',agent1, 'Score: ', a1_score)
        print('Agent2: ', agent2, 'score: ', a2_score)
print('\n')


Qlearner 10,000 Train Iters Game Results
---------------------------------

Agent1:  Random Score:  480
Agent2:  Random score:  520

Agent1:  Random Score:  9
Agent2:  Guru score:  991

Agent1:  Random Score:  258
Agent2:  Qlearner score:  742

Agent1:  Guru Score:  999
Agent2:  Random score:  1

Agent1:  Guru Score:  934
Agent2:  Guru score:  66

Agent1:  Guru Score:  994
Agent2:  Qlearner score:  6

Agent1:  Qlearner Score:  739
Agent2:  Random score:  261

Agent1:  Qlearner Score:  29
Agent2:  Guru score:  971

Agent1:  Qlearner Score:  929
Agent2:  Qlearner score:  71


CPU times: total: 438 ms
Wall time: 432 ms


## Q-learner is penalized for actions that do not result in a win.

In [5]:
%%time
##teach the q-learner by playing the game 
learn_num_games = 1000
play_num_games = 1000
explore_penalty = -15.0
nim_qlearn(learn_num_games, explore_penalty)

##iterate through agents for the first player to go
print('\nQlearner Explore Penalty Game Results')
print('---------------------------------')
for agent1 in agents:
    ##iterate through agents for the second player to go
    for agent2 in agents:
        a1_score, a2_score = play_games(play_num_games, agent1, agent2)
        
        ##record agent1 performance
        results['agent'].append(agent1)
        results['score'].append(a1_score)
        results['turn'].append('first')
        results['descr'].append('qlearner_with_penalty')
        
        ##record agent2 performance
        results['agent'].append(agent2)
        results['score'].append(a2_score)
        results['turn'].append('second')
        results['descr'].append('qlearner_with_penalty')
        
        print('\nAgent1: ',agent1, 'Score: ', a1_score)
        print('Agent2: ', agent2, 'score: ', a2_score)
print('\n')


Qlearner Explore Penalty Game Results
---------------------------------

Agent1:  Random Score:  512
Agent2:  Random score:  488

Agent1:  Random Score:  11
Agent2:  Guru score:  989

Agent1:  Random Score:  273
Agent2:  Qlearner score:  727

Agent1:  Guru Score:  997
Agent2:  Random score:  3

Agent1:  Guru Score:  942
Agent2:  Guru score:  58

Agent1:  Guru Score:  999
Agent2:  Qlearner score:  1

Agent1:  Qlearner Score:  724
Agent2:  Random score:  276

Agent1:  Qlearner Score:  12
Agent2:  Guru score:  988

Agent1:  Qlearner Score:  542
Agent2:  Qlearner score:  458


CPU times: total: 172 ms
Wall time: 160 ms


## Q-learner trained for 100,000 training iterations and penalized for actions that do not result in a win.

In [6]:
%%time
##teach the q-learner by playing the game 
learn_num_games = 10000
play_num_games = 1000
explore_penalty = -15
nim_qlearn(learn_num_games, explore_penalty)

##iterate through agents for the first player to go
print('\nQlearner with 10000 Training iterations and Explore Penalty Game Results')
print('---------------------------------')
for agent1 in agents:
    ##iterate through agents for the second player to go
    for agent2 in agents:
        a1_score, a2_score = play_games(play_num_games, agent1, agent2)
        
        ##record agent1 performance
        results['agent'].append(agent1)
        results['score'].append(a1_score)
        results['turn'].append('first')
        results['descr'].append('qlearner_penalty_increase_it')
        
        ##record agent2 performance
        results['agent'].append(agent2)
        results['score'].append(a2_score)
        results['turn'].append('second')
        results['descr'].append('qlearner_penalty_increase_it')
        
        print('\nAgent1: ',agent1, 'Score: ', a1_score)
        print('Agent2: ', agent2, 'score: ', a2_score)
print('\n')

df = pd.DataFrame(results)


Qlearner with 10000 Training iterations and Explore Penalty Game Results
---------------------------------

Agent1:  Random Score:  516
Agent2:  Random score:  484

Agent1:  Random Score:  6
Agent2:  Guru score:  994

Agent1:  Random Score:  299
Agent2:  Qlearner score:  701

Agent1:  Guru Score:  1000
Agent2:  Random score:  0

Agent1:  Guru Score:  949
Agent2:  Guru score:  51

Agent1:  Guru Score:  999
Agent2:  Qlearner score:  1

Agent1:  Qlearner Score:  734
Agent2:  Random score:  266

Agent1:  Qlearner Score:  26
Agent2:  Guru score:  974

Agent1:  Qlearner Score:  925
Agent2:  Qlearner score:  75


CPU times: total: 453 ms
Wall time: 441 ms


## Results

In [7]:
print('\nStatistics for agent that went first')
df[df['turn'] == 'first'].describe()


Statistics for agent that went first


,score
count,36.000000
mean,579.555556
std,384.456012
min,6.000000
25%,269.250000
50%,622.000000
75%,943.750000
max,1000.000000


In [8]:
print('\nStatistics for agent that went second')
df[df['turn'] == 'second'].describe()


Statistics for agent that went second


,score
count,36.000000
mean,420.444444
std,384.456012
min,0.000000
25%,56.250000
50%,378.000000
75%,730.750000
max,994.000000


In [9]:
print('\nQlearner Baseline Performance\n', df[(df.agent == 'Qlearner') & (df.descr == 'baseline')].mean())

print('\nQlearner Trained on 1,000,000 Iterations Performance\n', 
      df[(df.agent == 'Qlearner') & (df.descr == 'base_increased_train_it')].mean())

print('\nQlearner Exploration Penalty\n', 
      df[(df.agent == 'Qlearner') & (df.descr == 'qlearner_with_penalty')].mean())

print('\nQlearner Exploration Penalty and Increased Training Iterations\n', 
      df[(df.agent == 'Qlearner') & (df.descr == 'qlearner_penalty_increase_it')].mean())


Qlearner Baseline Performance
 score    398.5
dtype: float64

Qlearner Trained on 1,000,000 Iterations Performance
 score    419.333333
dtype: float64

Qlearner Exploration Penalty
 score    410.666667
dtype: float64

Qlearner Exploration Penalty and Increased Training Iterations
 score    410.333333
dtype: float64


Observations:

Note: Here score is defined as the number of games won by an agent, which is based on a 1000 game 

series.

- In general, agent's that went first in the game of Nim, scored higher about 28% higher than the 

agent that went second.


- The highest score for the agent that went first was 1000, which is a perfect score.


- The highest score for the agent that went second was 998, which is near perfect.


- The lowest score for the agent that went first was 2.0.


- The lowest score for the agent that went second was 0.


- The Qlearner's mean-average score improved by about 5.0% by increasing the training iterations to

  10,000 instead of 1000.
  
 
 - The Qlearner's mean-average score improved by about 3.0% by introducing a exploration penalty of    
   negative 15.0 for the agent's actions that do not result in a win.
   
 
 - The Qlearner's mean-average score improved by about 3.0% by introducing an exploration penalty of    
   negative 15.0 for the agent's actions that do not result in a win and also increasing the agent's    
   training iterations to 10k.
   
   
   
   Summary:

Of the three hypothesis made at the beginning of this assignment, only the first two were correct. 

First, increasing the training iterations helped the Qlearner increase performance by increasing the 

number of games won. Second, adding a penalty of negative 15.0 for when the agent took an action 

that did not result in a win. Though the third case of adding the penalty and increasing training 

iterations outperformed the baseline case, it did not do better then increasing train iterations 

alone or penalizing the agent for exploring the environment alone.


The learning rate (alpha) and discount factor (gamma) were also experimented with to a small extent 

to see if either hyperparameter would make a significant difference. The values settled on were 

alpha and gamma of 0.9 and 0.8 respectively. Something that might also help performance would be an 

optimal value search for these two hyperparameters. This assignment has been much different then the 

previous assignments, or atleast the concepts are much different then supervised vs unsupervised 

learning. 